# Transformation of Natural Language Questions to SQL Queries within the Healthcare Domain Using the T5 Model

## Track of GPU Activity

In [1]:
!nvidia-smi

Sun Sep 19 16:11:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Data Preparation

In [2]:
# Import essential libraries for data preparation

import json
import pandas as pd
import numpy as np

In [3]:
# Load the training data to get insight

with open("../input/train-data/train.json") as folder:
    train_data = json.loads("[" + 
        folder.read().replace("}\n{", "},\n{") + 
    "]")

In [4]:
# Check the number of rows
len(train_data)

8000

In [5]:
train_data[0].keys()

dict_keys(['key', 'format', 'question_refine', 'sql', 'question_refine_tok', 'sql_tok'])

In [6]:
train_data[0]

{'key': '1f4675717f61aa61ed79e4b4deb69e76',
 'format': {'table': [0],
  'cond': [[0, 7, 0, 'HAIT']],
  'agg_col': [[0, 0]],
  'sel': 1},
 'question_refine': 'find the number of patients who prefer haitian language.',
 'sql': 'SELECT COUNT ( DISTINCT DEMOGRAPHIC."SUBJECT_ID" ) FROM DEMOGRAPHIC WHERE DEMOGRAPHIC."LANGUAGE" = "HAIT"',
 'question_refine_tok': ['find',
  'the',
  'number',
  'of',
  'patients',
  'who',
  'prefer',
  'haitian',
  'language',
  '.'],
 'sql_tok': ['SELECT',
  'COUNT',
  '(',
  'DISTINCT',
  'DEMOGRAPHIC."SUBJECT_ID"',
  ')',
  'FROM',
  'DEMOGRAPHIC',
  'WHERE',
  'DEMOGRAPHIC."LANGUAGE"',
  '=',
  '"HAIT"']}

In [7]:
train_data[0]["question_refine"]

'find the number of patients who prefer haitian language.'

In [8]:
train_data[0]["sql"]

'SELECT COUNT ( DISTINCT DEMOGRAPHIC."SUBJECT_ID" ) FROM DEMOGRAPHIC WHERE DEMOGRAPHIC."LANGUAGE" = "HAIT"'

In [9]:
# Function to extract questions and sql queries, returns pandas dataframe

def extract_questions_and_sql(path):
    with open(path) as folder:
        sql_question_data = json.loads("[" + 
            folder.read().replace("}\n{", "},\n{") + 
        "]")
    
    data_rows = []
    
    for question_and_sql in sql_question_data:
        question = question_and_sql["question_refine"]
        sql_query = question_and_sql["sql"]
        
        data_rows.append({
            "question": question,
            "sql_query": sql_query
        })
    
    return pd.DataFrame(data_rows)

### Training Data

In [10]:
# Load the training data using the extract_questions_and_sql function
train_df = extract_questions_and_sql("../input/train-data/train.json")

In [11]:
# Check the first 5 rows
train_df.head()

,question,sql_query
0,find the number of patients who prefer haitian...,"SELECT COUNT ( DISTINCT DEMOGRAPHIC.""SUBJECT_I..."
1,give me the number of patients whose religion ...,"SELECT COUNT ( DISTINCT DEMOGRAPHIC.""SUBJECT_I..."
2,count the number of patients whose ethnicity i...,"SELECT COUNT ( DISTINCT DEMOGRAPHIC.""SUBJECT_I..."
3,how many patients are diagnosed with the prima...,"SELECT COUNT ( DISTINCT DEMOGRAPHIC.""SUBJECT_I..."
4,What number of patients were primarily diagnos...,"SELECT COUNT ( DISTINCT DEMOGRAPHIC.""SUBJECT_I..."


In [12]:
# Check the total number of rows and columns
train_df.shape

(8000, 2)

In [13]:
# Check for any missing values
train_df.isna().sum()

question     0
sql_query    0
dtype: int64

In [14]:
# Check for the total number of duplicated rows
train_df.duplicated().sum()

0

In [15]:
# Check for the total number of unique questions
len(train_df.question.unique())

7998

In [16]:
# Check for the total number of unique SQL queries
len(train_df.sql_query.unique())

8000

In [17]:
# Check the rows of the duplicated questions
train_df[train_df.question.duplicated(keep=False)]

,question,sql_query
2587,count the number of patients who have stayed i...,"SELECT COUNT ( DISTINCT DEMOGRAPHIC.""SUBJECT_I..."
4755,count the number of patients who have stayed i...,"SELECT COUNT ( DISTINCT DEMOGRAPHIC.""SUBJECT_I..."
7311,specify icd9 code for patient id 6983,"SELECT DIAGNOSES.""SHORT_TITLE"" FROM DIAGNOSES ..."
7315,specify icd9 code for patient id 6983,"SELECT PROCEDURES.""LONG_TITLE"" FROM PROCEDURES..."


In [18]:
# Remove duplicated rows
train_df.drop(train_df.index[[2587,4755,7311,7315]], inplace=True)

In [19]:
# Final check for the total number of unique questions
len(train_df.question.unique())

7996

### Validation Data 

In [20]:
# Load the validation data using the extract_questions_and_sql function
validation_df = extract_questions_and_sql("../input/validation-data/dev.json")

In [21]:
# Check the first 5 rows
validation_df.head()

,question,sql_query
0,count the number of patients whose diagnoses s...,"SELECT COUNT ( DISTINCT DEMOGRAPHIC.""SUBJECT_I..."
1,which patients have seroma complicating a proc...,"SELECT COUNT ( DISTINCT DEMOGRAPHIC.""SUBJECT_I..."
2,give the number of patients diagnosed with mal...,"SELECT COUNT ( DISTINCT DEMOGRAPHIC.""SUBJECT_I..."
3,how many patients have been diagnosed with oli...,"SELECT COUNT ( DISTINCT DEMOGRAPHIC.""SUBJECT_I..."
4,which patients had cardiopulmonary resuscitati...,"SELECT COUNT ( DISTINCT DEMOGRAPHIC.""SUBJECT_I..."


In [22]:
# Check the total number of rows and columns
validation_df.shape

(1000, 2)

In [23]:
# Check for any missing values
validation_df.isna().sum()

question     0
sql_query    0
dtype: int64

In [24]:
# Check for the total number of duplicated rows
validation_df.duplicated().sum()

0

In [25]:
# Check for the total number of unique questions
len(validation_df.question.unique())

1000

In [26]:
# Check for the total number of unique SQL queries
len(validation_df.sql_query.unique())

1000

### Test Data

In [27]:
# Load the test data using the extract_questions_and_sql function
test_df = extract_questions_and_sql("../input/test-data/test.json")

In [28]:
# Check the first 5 rows
test_df.head()

,question,sql_query
0,how many patients were born before the year 2060?,"SELECT COUNT ( DISTINCT DEMOGRAPHIC.""SUBJECT_I..."
1,how many patients had the diagnosis icd9 code ...,"SELECT COUNT ( DISTINCT DEMOGRAPHIC.""SUBJECT_I..."
2,let me know the number of patients who have di...,"SELECT COUNT ( DISTINCT DEMOGRAPHIC.""SUBJECT_I..."
3,what is the total number of patiemts who had c...,"SELECT COUNT ( DISTINCT DEMOGRAPHIC.""SUBJECT_I..."
4,give me the number of patients who have been d...,"SELECT COUNT ( DISTINCT DEMOGRAPHIC.""SUBJECT_I..."


In [29]:
# Check the total number of rows and columns
test_df.shape

(1000, 2)

In [30]:
# Check for any missing values
test_df.isna().sum()

question     0
sql_query    0
dtype: int64

In [31]:
# Check for the total number of duplicated rows
test_df.duplicated().sum()

0

In [32]:
# Check for the total number of unique questions
len(test_df.question.unique())

1000

In [33]:
# Check for the total number of unique SQL queries
len(test_df.sql_query.unique())

1000

## Modelling

In [34]:
# Import essential libraries for the modelling section

import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5TokenizerFast as T5Tokenizer
)

In [35]:
# Random seed
pl.seed_everything(42)

42

In [36]:
modelName = "t5-base"

In [37]:
# Download the T5 tokenizer from the pretained T5 base model 
tokenizer = T5Tokenizer.from_pretrained(modelName)

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [38]:
# Class to create a dataset that is extended from PyTorch dataset
class MimicSQLDataset(Dataset):
    # Create a constructer
    def __init__(
        self,
        # The data as pandas DataFrame
        data: pd.DataFrame,
        # The tokenizer
        tokenizer: T5Tokenizer,
        # maximum token length of the source text which is the question 
        source_max_token_len: int = 256,
        # maximum token length of the target text which is the SQL query 
        target_max_token_len: int = 128
    ):
        # Store everything
        self.tokenizer = tokenizer
        self.data = data
        self.source_max_token_len = source_max_token_len
        self.target_max_token_len = target_max_token_len
        
    # The __len__ (length) method
    def __len__(self):
        return len(self.data)
    
    # The __getitem__ method
    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]
        
        # Source encoding - the natural questions
        source_encoding = tokenizer(
            data_row["question"],
            # Set maximum length as the maximum token length of the source text
            max_length = self.source_max_token_len,
            padding = "max_length",
            truncation = "only_first",
            add_special_tokens = True,
            return_attention_mask = True,
            return_tensors = "pt"
        )
        
        # Target encoding - the SQL query
        target_encoding = tokenizer(
            data_row["sql_query"],
            # Set maximum length as the maximum token length of the target text
            max_length = self.target_max_token_len,
            padding = "max_length",
            truncation = True,
            add_special_tokens = True,
            return_attention_mask = True,
            return_tensors = "pt"
        
        )
        
        # Create the labels
        labels = target_encoding["input_ids"]
        # Convert the ignored labels to -100 to be excluded from computation
        labels[labels == 0] = -100
        
        return dict(
            question = data_row["question"],
            sql_query = data_row ["sql_query"],
            input_ids = source_encoding["input_ids"].flatten(),
            attention_mask = source_encoding["attention_mask"].flatten(),
            labels = labels.flatten()
        )

In [40]:
# Class to create a Lightning Data Module
class MimicSQLDataModule(pl.LightningDataModule):
    # Create a constructer
    def __init__(
        self,
        # The train data as pandas DataFrame
        train_df: pd.DataFrame,
        # The validation data as pandas DataFrame
        validation_df: pd.DataFrame,
        # The test data as pandas DataFrame
        test_df: pd.DataFrame,
        # The tokenizer
        tokenizer: T5Tokenizer,
        # The batch size
        batch_size: int = 16,
        # maximum token length of the source text which is the question
        source_max_token_len: int = 256,
        # maximum token length of the target text which is the SQL query
        target_max_token_len: int = 128
    ):
        super().__init__()
        # Store everything
        self.batch_size = batch_size
        self.train_df = train_df
        self.validation_df = validation_df
        self.test_df = test_df
        self.tokenizer = tokenizer
        self.source_max_token_len = source_max_token_len
        self.target_max_token_len = target_max_token_len
    
    # The setup method
    def setup(self):
        self.train_dataset = MimicSQLDataset(
            self.train_df,
            self.tokenizer,
            self.source_max_token_len,
            self.target_max_token_len
        )
        
        self.validation_dataset = MimicSQLDataset(
            self.validation_df,
            self.tokenizer,
            self.source_max_token_len,
            self.target_max_token_len
        )
        
        self.test_dataset = MimicSQLDataset(
            self.test_df,
            self.tokenizer,
            self.source_max_token_len,
            self.target_max_token_len
        )
    
    # PyTorch data loaders to save memory and boost up the speed.
    
    # Train data loader to parallelise the process of data loading with automatic batching.
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            num_workers = 4,
            shuffle = True,
            batch_size = self.batch_size
        )
    
    # Validation data loader to parallelise the process of data loading with automatic batching.
    def val_dataloader(self):
        return DataLoader(
            self.validation_dataset,
            num_workers = 4,
            batch_size = self.batch_size
        ) 
    
    # Test data loader to parallelise the process of data loading with automatic batching.
    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            num_workers = 4,
            batch_size = self.batch_size
        )

In [42]:
# Batch size
BATCH_SIZE = 16
# Number of epochs
NUM_EPOCHS = 14

data_module = MimicSQLDataModule(train_df, validation_df, test_df, tokenizer, batch_size = BATCH_SIZE)
data_module.setup()

In [44]:
# Class to creates a PyTorch Lightning Module 
class MimicSQLModel(pl.LightningModule):
    
    # Create a constructer
    def __init__(self):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(modelName, return_dict = True)
    
    # The forward method 
    def forward(self, input_ids, attention_mask, labels = None):
        output = self.model(
            input_ids = input_ids,
            attention_mask = attention_mask,
            labels = labels
        )
        
        return output.loss, output.logits
    
    # The trainig step
    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("train_loss", loss, prog_bar = True, logger = True)
        return loss 
    
    # The validation step
    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("val_loss", loss, prog_bar = True, logger = True)
        return loss 
    
    # the test step
    def test_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("test_loss", loss, prog_bar = True, logger = True)
        return loss 
    
    # The optomizer and learning rate
    def configure_optimizers(self):
        return AdamW(self.parameters(), lr = 0.0005)

In [46]:
# Save the MimicSQLModel() as the model
model = MimicSQLModel()

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [47]:
# Create a checkpoint callback  
checkpoint_callback = ModelCheckpoint(
    dirpath = "checkpoints",
    filename = "best-checkpoint",
    save_top_k = 1,
    verbose = True,
    monitor = "val_loss",
    mode = "min"
)

In [48]:
# PyTorch Lightning trainer for training automation
trainer = pl.Trainer(
    checkpoint_callback = checkpoint_callback,
    max_epochs = NUM_EPOCHS,
    gpus = 1,
    progress_bar_refresh_rate = 30
)

In [49]:
trainer.fit(model, data_module)

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

## Evaluation

In [50]:
# Import essential libraries for the evaluation section

from random import randrange
from sklearn import metrics
from fuzzywuzzy import fuzz

In [51]:
# Switch the model out of the training mode
model.eval()

MimicSQLModel(
  (model): T5ForConditionalGeneration(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseReluDense(
                (wi): Linear(in_features=768, out_features=3072, bias=False)
                (wo): Linear(in_feat

In [52]:
# Function that takes a question and predicts the corresponding SQl query
def sql_generator(question):
    source_encoding = tokenizer(
        question["question"],
        max_length = 256,
        padding = "max_length",
        truncation = "only_first",
        return_attention_mask = True,
        add_special_tokens  = True,
        return_tensors = "pt"
    )
    
    ids_generated = model.model.generate(
        # The input ids of the source encoding 
        input_ids=source_encoding["input_ids"],
        # The attention mask of the source encoding
        attention_mask=source_encoding["attention_mask"],
        # greedy search
        num_beams=1,
        max_length=180,
        repetition_penalty=2.5,
        early_stopping=True,
        use_cache=True)
    
    predictions = [
        tokenizer.decode(id_generated, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        for id_generated in ids_generated
    ]
    
    return "".join(predictions)

### Evaluation on the validation data

In [54]:
# Generate randomly 5 questions along with their corresponding actual and predicted SQL queries.
for numbers in range(5):
    nums = []
    nums.append(randrange(0, 1000))
    for num in nums:
        random_question = validation_df.iloc[num]
        print(random_question["question"])
        print("\nActual SQL query: %s" % random_question["sql_query"])
        print("Predicted SQL query: %s" % sql_generator(random_question))
        print("--------------------------------------------------------------------------------\n")

provide me the number of patients who were hospitalized for more than a day and were less than 89 years of age.

Actual SQL query: SELECT COUNT ( DISTINCT DEMOGRAPHIC."SUBJECT_ID" ) FROM DEMOGRAPHIC WHERE DEMOGRAPHIC."AGE" < "89" AND DEMOGRAPHIC."DAYS_STAY" > "1"
Predicted SQL query: SELECT COUNT ( DISTINCT DEMOGRAPHIC."SUBJECT_ID" ) FROM DEMOGRAPHIC WHERE DEMOGRAPHIC."AGE"  "89" AND DEMOGRAPHIC."DAYS_STAY" > "1"
--------------------------------------------------------------------------------

What is the number of patients admitted before year 2139 whose procedure long title is other closed [endoscopic] biopsy of biliary duct or sphincter of oddi?

Actual SQL query: SELECT COUNT ( DISTINCT DEMOGRAPHIC."SUBJECT_ID" ) FROM DEMOGRAPHIC INNER JOIN PROCEDURES on DEMOGRAPHIC.HADM_ID = PROCEDURES.HADM_ID WHERE DEMOGRAPHIC."ADMITYEAR" < "2139" AND PROCEDURES."LONG_TITLE" = "Other closed [endoscopic] biopsy of biliary duct or sphincter of Oddi"
Predicted SQL query: SELECT COUNT ( DISTINCT DEMO

In [56]:
# Make a list of correct SQL queries and a list of predicted SQL queries.
correct_query = []
predicted_query = []
for i in range(1000):
    question = validation_df.iloc[i]
    correct_query.append(question["sql_query"])
    predicted_query.append(sql_generator(question))

In [58]:
# Exact match accuracy
metrics.accuracy_score(correct_query, predicted_query)

0.457

In [62]:
# Apply fuzz.ratio and make a list of fuzz ratio scores
results = []
for (correct,predicted) in zip(correct_query, predicted_query):
    results.append(fuzz.ratio(correct,predicted))

In [63]:
# Average score of fuzz ratio
sum(results)/len(results)

97.952

### Evaluation on the test data

In [64]:
# Generate randomly 5 questions along with their corresponding actual and predicted SQL queries.
for numbers in range(5):
    nums = []
    nums.append(randrange(0, 1000))
    for num in nums:
        random_question = test_df.iloc[num]
        print(random_question["question"])
        print("\nActual SQL query: %s" % random_question["sql_query"])
        print("Predicted SQL query: %s" % sql_generator(random_question))
        print("--------------------------------------------------------------------------------\n")

what is the diagnosis of Tracy Farmer?

Actual SQL query: SELECT DIAGNOSES."SHORT_TITLE" FROM DEMOGRAPHIC INNER JOIN DIAGNOSES on DEMOGRAPHIC.HADM_ID = DIAGNOSES.HADM_ID WHERE DEMOGRAPHIC."NAME" = "Tracy Farmer"
Predicted SQL query: SELECT DIAGNOSES."LONG_TITLE" FROM DEMOGRAPHIC INNER JOIN DIAGNOSES on DEMOGRAPHIC.HADM_ID = DIAGNOSES.HADM_ID WHERE DEMOGRAPHIC."NAME" = "Tracy Farmer"
--------------------------------------------------------------------------------

provide me the procedure icd9 code of patient with patient id 1875.

Actual SQL query: SELECT PROCEDURES."ICD9_CODE" FROM PROCEDURES WHERE PROCEDURES."SUBJECT_ID" = "1875"
Predicted SQL query: SELECT PROCEDURES."ICD9_CODE" FROM PROCEDURES WHERE PROCEDURES."SUBJECT_ID" = "1875"
--------------------------------------------------------------------------------

Is the subject id 74032 married, find his religion ?

Actual SQL query: SELECT DEMOGRAPHIC."MARITAL_STATUS",DEMOGRAPHIC."RELIGION" FROM DEMOGRAPHIC WHERE DEMOGRAPHIC."SUBJE

In [65]:
# Make a list of correct SQL queries and a list of predicted SQL queries.
correct_query = []
predicted_query = []
for i in range(1000):
    question = test_df.iloc[i]
    correct_query.append(question["sql_query"])
    predicted_query.append(sql_generator(question))

In [66]:
# Exact match accuracy
metrics.accuracy_score(correct_query, predicted_query)

0.593

In [68]:
# Apply fuzz.ratio and make a list of fuzz ratio scores
results = []
for (correct,predicted) in zip(correct_query, predicted_query):
    results.append(fuzz.ratio(correct,predicted))

In [69]:
#accuracy using fuzz.ratio
sum(results)/len(results)

98.506